# Computer Assignment 2, Genetic Algorithms

### In this project, our goal is to use genetic algorithms to select the gates in a logic circuit in such a way that by applying the inputs, we get the corresponding output according to the truth table.

## Reading Data
The input is as follows:
The truth_table.csv file contains n columns, the first n-1 columns are the inputs of the logic circuit and the last column is the output. In each output row, it represents the output of the circuit if the inputs in the same row are applied.

In [1]:
import csv

def read_input_file():
    data = []
    with open('truth_table.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line = 0
        for row in csv_reader:
            if line != 0:
                data.append(row)
            line += 1
    return data

In [2]:
def initial_samples(data):
    input_samples = []
    output_samples = []
    for sample in data:
        input_samples.append(sample[:-1])
        output_samples.append(sample[-1])
    return input_samples, output_samples, len(input_samples), len(input_samples[0])-1

## Logical gates

The gates in question are And, Or, Xor, Nand, Nor and Xnor, whose functionality is implemented in the Operator class, and for ease of implementation, they correspond to the numbers 1, 2, 3, 4, 5, and 6, respectively.

In [3]:
class Operator:
    @staticmethod
    def and_(a , b):
        return a and b
    @staticmethod
    def or_(a , b):
        return a or b
    @staticmethod
    def xor_(a , b):
        return a ^ b
    @staticmethod
    def nand_(a , b):
        return not (a and b)
    @staticmethod
    def nor_(a , b):
        return not (a or b)
    @staticmethod
    def xnor_(a , b):
        return not (a ^ b)

def initial_gates():
    gates = {
        1: Operator.and_,
        2: Operator.or_,
        3: Operator.xor_,
        4: Operator.nand_,
        5: Operator.nor_,
        6: Operator.xnor_,
    }
    return gates

## Circuit

The following function calculates the output according to the order of the gates and according to the input of circuit.

In [4]:
def get_circuit_output(getes_num, inputs):
    out = 0
    in1 = get_boolean(inputs[0])
    in_index = 1
    for num in getes_num :
        in2 = get_boolean(inputs[in_index])
        out = gates[num](in1, in2)
        in1 = out 
        in_index += 1
    return out

def get_boolean(s):
    if s == 'TRUE':
        return True
    elif s == 'FALSE':
        return False

## Phase 1, Basic concepts

### Gene
Here, each gate used in the circuit represents a gene.

### Chromosomr
A chromosome is a string of genes, since in this case the gates represent genes, the chromosome is a string of gates whose order is important. Because the components of the circuit are arranged in the order said in the project.

In [5]:
class Chromosome :
    def __init__(self, genes):
        self.genes = genes

    def fitness(self):
        success = 0
        for i in range(number_of_samples):
            if get_circuit_output(self.genes, input_samples[i]) == get_boolean(output_samples[i]):
                success += 1
        return success/number_of_samples

## Phase 2, Generate base population

Since the generation of the initial population must be random. Here we produce POPULATION_SIZE chromosomes with completely random genes.
The number of people in the initial population is a hyper parameter and should be chosen depending on the type of problem. The larger number of people, the longer length of a loop, and the smaller number, the higher number of loops.

In [7]:
import numpy as np

POPULATION_SIZE = 50

def generate_basic_population():
    population = []
    new_population_genes = np.random.choice(range(1,7), size=(POPULATION_SIZE, number_of_genes)).tolist()
    for genes in new_population_genes :
        population.append(Chromosome(genes))
    return population

## Phase 3, Fitness

Each chromosome has a fitness function. To calculate it, we first place the gates, which are chromosome genes, in the circuit in order, and then obtain the output of the circuit for all the input strings. Chromosome fitness value is the division of the correct number of outputs by the total output.

## Phase 4, Selection, Crossover, Mutation and next population

### Selection

In [9]:
import random

def calculate_all_fitnesses(mating_pool):
    fitnesses = []
    max_fitness = 0
    for chromosome in mating_pool:
        fitness = chromosome.fitness()
        if fitness > max_fitness:
            max_fitness = fitness
        fitnesses.append((chromosome, chromosome.fitness()))
    return sorted(fitnesses, key=lambda item: item[1]), max_fitness

#### New Idea

Here, after sorting the chromosomes according to their fitness value, we pass on the number of the ones that have the highest amount of fitness directly to the next generation, because it makes the maximum amount of fitness in each loop does not decrease and we force it to Reach the result sooner.

In [43]:
DIRECT_GENERATION_SIZE = int(0.16 * POPULATION_SIZE)

#### 1 - Rank base selection

In this type, we first calculate the fitness value of all chromosomes. Then we sort these values in ascending order. The probability of the presence of each chromosome in the next generation is equal to dividing its rank by the total number of chromosomes.

In [10]:
def rank_base_selection(mating_pool):
    direct_generation = []
    fitnesses, max_fitness = calculate_all_fitnesses(mating_pool)
    for i in range(DIRECT_GENERATION_SIZE):
        direct_generation.append(fitnesses[POPULATION_SIZE - DIRECT_GENERATION_SIZE + i][0])
    probablities = [x / POPULATION_SIZE for x in range(1, POPULATION_SIZE + 1)]
    return random.choices([i[0] for i in fitnesses], weights=probablities, k=POPULATION_SIZE - DIRECT_GENERATION_SIZE), max_fitness, direct_generation

#### 2 - Fitness proportionate election

In this type, we first calculate the fitness value of all chromosomes. Then we sort these values in ascending order. The probability of the presence of each chromosome in the next community is equal to dividing its fitness value by the sum of the total fitness.

In [11]:
def proportionate_selection(mating_pool):
    direct_generation = []
    fitnesses, max_fitness = calculate_all_fitnesses(mating_pool)
    for i in range(DIRECT_GENERATION_SIZE):
        direct_generation.append(fitnesses[POPULATION_SIZE - DIRECT_GENERATION_SIZE + i][0])
    sum_of_fitnesses = 0
    for fitness in fitnesses:
        sum_of_fitnesses += fitness[1]
    probablities = [fitness[1] / sum_of_fitnesses for fitness in fitnesses]
    return random.choices([i[0] for i in fitnesses], weights=probablities, k=POPULATION_SIZE - DIRECT_GENERATION_SIZE), max_fitness, direct_generation

### Crossover

In [13]:
def mix_genes(points, chromosome1, chromosome2):
    child1 = []
    child2 = []
    parent1 = chromosome1.genes
    parent2 = chromosome2.genes
    temp = []
    last_point = 0

    for point in points:
        for gene in parent2[last_point:point]: child1.append(gene)
        for gene in parent1[last_point:point]: child2.append(gene)
        
        last_point = point
        temp = parent1
        parent1 = parent2
        parent2 = temp

    return Chromosome(child1), Chromosome(child2)

#### 1 - N point crossover

In this type, the genes on both chromosomes are broken down into n+1 parts by n points, and these parts move between each other and make children.

In [14]:
def n_point_cross_over(n, chromosome1, chromosome2):
    points = np.random.choice(range(1, number_of_genes), replace=False, size=n).tolist()
    points.append(number_of_genes)
    return mix_genes(sorted(points), chromosome1, chromosome2)

#### 2 - Uniform crossover

In this type, the number of points is not constant and it is randomly decided which parts of the two strings are broken.

In [15]:
def uniform_cross_over(chromosome1, chromosome2):
    points = []
    for i in range(1, number_of_genes):
        is_point = np.random.choice((True, False)).tolist()
        if is_point: points.append(i)
    points.append(number_of_genes)
    return mix_genes(sorted(points), chromosome1, chromosome2)

#### Probablity of Crossover

Crossover is performed between two parents with PC probability.

In [25]:
PC = 0.6

def decide_to_do_cross_over(chromosome1, chromosome2):
    do_cross_over = np.random.choice([True, False], p=[PC, 1-PC])
    if do_cross_over:
        return n_point_cross_over(2, chromosome1, chromosome2)
    return chromosome1, chromosome2

### Mutation

Each of the parent genes is replaced with another gene to produce a child with PM probability. Selection of alternative gene is random.

In [36]:
PM = 0.1

def flip_gene(gene_num):
    remain_genes = [x for x in gates.keys() if x != gene_num]
    final_gene = np.random.choice(remain_genes)
    return final_gene

def mutation(chromosome):
    for i in range(number_of_genes):
        do_flip = np.random.choice([True, False], p=[PM, 1-PM])
        if do_flip :
            chromosome.genes[i] = flip_gene(chromosome.genes[i])
    return chromosome

### Next population

In general, the following happens in each loop of the genetic algorithm:
- Sort chromosomes according to the amount of fitness and generate mating pool from them according to one of the Selection algorithms
- Shuffle the mating pool
- Apply Crossover algorithm between pairs of chromosomes
- Apply Mutation algorithm on each chromosome

#### Initial values

In [22]:
def initial_samples(data):
    input_samples = []
    output_samples = []
    for sample in data:
        input_samples.append(sample[:-1])
        output_samples.append(sample[-1])
    return input_samples, output_samples, len(input_samples), len(input_samples[0])-1

def initial_gates():
    gates = {
        1: Operator.and_,
        2: Operator.or_,
        3: Operator.xor_,
        4: Operator.nand_,
        5: Operator.nor_,
        6: Operator.xnor_,
    }
    return gates

In [23]:
data = read_input_file()
input_samples, output_samples, number_of_samples, number_of_genes = initial_samples(data)
gates = initial_gates()

#### Genetic Algorithm

In [20]:
import copy

MAX_REPEAT = 150
SAME_FITNESS_LIMIT = POPULATION_SIZE/2

def genetic_algorithm():
    population = generate_basic_population()
    max_fitness = 0
    repeat = 0
    result = 0
    number_of_same_fitness = 0
    last_fitness = 0
    while max_fitness != 1 and  repeat != MAX_REPEAT:
        if number_of_same_fitness == SAME_FITNESS_LIMIT:
            population = generate_basic_population()
        repeat += 1
        mating_pool, max_fitness, direct_generation = rank_base_selection(population)
        if max_fitness == last_fitness:
            number_of_same_fitness += 1
        else:
            number_of_same_fitness = 0
        print(max_fitness)
        new_generation = direct_generation
        if max_fitness > result:
            result = max_fitness
        random.shuffle(mating_pool)
        for i in range(0, POPULATION_SIZE - DIRECT_GENERATION_SIZE, 2):
            chromosome1, chromosome2 = decide_to_do_cross_over(mating_pool[i], mating_pool[i+1])
            new_generation.append(mutation(copy.deepcopy(chromosome1)))
            new_generation.append(mutation(copy.deepcopy(chromosome2)))
        population = new_generation
        last_fitness = max_fitness
    return result, repeat

In [44]:
import time

start = time.time()
print(genetic_algorithm(),  time.time() - start)

0.8671875
0.90625
0.90625
0.90625
0.9375
0.9375
0.94921875
0.953125
0.9609375
0.96875
0.96875
0.96875
0.96875
0.96875
0.96875
0.984375
0.984375
0.984375
1.0
(1.0, 19) 8.892204999923706


## Phase 5, Questions

<ol>
<li> The fitness must have an accurate estimate of the quality of the solution. Here again, because we want to finally get the order of the gates to print the correct output for all input strings, we take the fitness as the percentage of correct output.
Although the fitness function should not only correlate closely with the designer's goal, but it should also be computationally efficient. Speed of execution is very important, as a typical genetic algorithm must be iterated many times in order to produce a usable result for a non-trivial problem.

 <li> Ranke base selection is used here because in the other one (FPS) when the fitness value of one chromosome is much higher than the next chromosomes, it is much more likely to be selected, which means that some good and helpful genes are not used.

<li> N point crossover was used here to implement crossover because we did not want to make too many changes.
In general, the crossover algorithm is used to create children by combining two parents who have not been able to get us to the correct answer.
And the mutation algorithm is used to change existing genes to other genes in the hope of improving performance. The probability is usually considered a small number so that population converges more quickly.

<li> When the best answer we have in several consecutive loops remains constant, it means that we have reached a state where we do not get a better answer despite the combinations we make, so the initial population is probably not suitable or has not gone well due to the changes. In these cases, we can randomly generate a population again and continue.
This is likely to happen, especially here when the best people go direct to the next generation.
In other words, from one place to another, more people in population become like each other while they are far from the right answer(local max). So with a small change, the genes do not get better, and it is best to change population.